영양 상태를 기반으로 영양제 추천 -> 식단 추천으로 얼마든지 변경이 가능하다는 생각

Cosine similarity 최대화 방향성 -> 이친구가 더 좋다는듯?
SSE(편차 제곱) 최소화 방향성 -> ㄴㄴ

어떤 식으로 다가가야 할 지 고민해야 합니다.

-> 예시를 들어서
        
        표준화 모델이니까 반반떄리다 치고

        1. 남은 영양성분

        2. 먹은 아점저 정보 (식단관리창에서)

        3. 아/점/저 -> 33% 해가지고 이렇게 하자? ㅇㅋ 

        4. 건강한 식단을 위해서는 아침>점심>저녁 인데 이걸 어캐 표현을 하나?

        5. 머신러닝 뭘로 할지? 
        - Cosine similarity인데....        

기본 식 -> 일일 활동량 비교해서 만듬 -> DB에서 해줄꺼고
음식영양성분 데이터 있으니까 여기서 URL을 담아둔 친구들을 사용해야 합니다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import re
import sys

In [3]:
%cd /content/drive/MyDrive/yolov7

/content/drive/MyDrive/yolov7


In [16]:
data = pd.read_excel('/content/drive/MyDrive/yolov7/food_data/food_info_가라로직.xlsx')
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,식품군,제조사명,지역명,중량,열량(kcal),탄수화물(g),단백질(g),지방(g),당류(g),나트륨(mg),콜레스트롤(mg),포화지방산(g),트랜스지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,NaN,NaN,충주,500.0,368.80,39.70,33.50,8.50,16.90,1264.31,106.18,1.90,0.10
1,2,닭갈비,NaN,NaN,NaN,NaN,NaN,전국(대표),400.0,595.61,44.90,45.90,25.80,21.20,1535.83,193.40,6.00,0.20
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,춘천,300.0,558.47,23.10,45.50,31.60,8.50,1016.94,217.67,11.30,0.30
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,NaN,NaN,전국(대표),70.0,176.72,13.35,11.56,8.57,3.15,286.91,31.44,2.32,0.04
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,NaN,NaN,전국(대표),100.0,184.00,31.10,3.10,5.20,11.60,743.37,0.12,0.80,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90603,90604,"코코아, 가루",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,268.00,43.63,22.85,20.06,0.49,24.00,0.00,12.11,0.01
90604,90605,"홍삼차, 액상",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,40.00,9.27,0.28,0.24,6.32,4.00,0.00,0.00,0.00
90605,90606,"홍차, 복숭아, 가루",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,397.00,98.12,0.32,0.38,42.31,41.00,0.00,0.00,0.00
90606,90607,"마요네즈, 저칼로리",NaN,NaN,NaN,조미료류,NaN,전국(대표),100.0,341.00,12.35,0.89,31.97,3.56,600.00,23.72,3.96,0.22


In [17]:
data.head(3)

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,식품군,제조사명,지역명,중량,열량(kcal),탄수화물(g),단백질(g),지방(g),당류(g),나트륨(mg),콜레스트롤(mg),포화지방산(g),트랜스지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,NaN,NaN,충주,500.0,368.80,39.7,33.5,8.5,16.9,1264.31,106.18,1.9,0.1
1,2,닭갈비,NaN,NaN,NaN,NaN,NaN,전국(대표),400.0,595.61,44.9,45.9,25.8,21.2,1535.83,193.40,6.0,0.2
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,춘천,300.0,558.47,23.1,45.5,31.6,8.5,1016.94,217.67,11.3,0.3


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90608 entries, 0 to 90607
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FOOD_SEQ   90608 non-null  int64  
 1   음식명        90608 non-null  object 
 2   레시피        828 non-null    object 
 3   한중일양       828 non-null    object 
 4   밥국반메면      754 non-null    object 
 5   식품군        82904 non-null  object 
 6   제조사명       85289 non-null  object 
 7   지역명        90608 non-null  object 
 8   중량         90608 non-null  float64
 9   열량(kcal)   90608 non-null  float64
 10  탄수화물(g)    90608 non-null  float64
 11  단백질(g)     90608 non-null  float64
 12  지방(g)      90608 non-null  float64
 13  당류(g)      90608 non-null  float64
 14  나트륨(mg)    90608 non-null  float64
 15  콜레스트롤(mg)  90608 non-null  float64
 16  포화지방산(g)   90608 non-null  float64
 17  트랜스지방(g)   90608 non-null  float64
dtypes: float64(10), int64(1), object(7)
memory usage: 12.4+ MB


In [19]:
# 필요없는 부분 -> Food_group, Food_mf 등등 삭제
# 쓸 부분 - seq, name, recipe, weigth, calor, crb, protein, fat 씁니다.
# 컬럼 제거
data.drop(
    ['식품군','제조사명','지역명','당류(g)','나트륨(mg)','콜레스트롤(mg)'
    ,'포화지방산(g)','트랜스지방(g)'],
    axis=1,
    errors='ignore',
    inplace=True
)

In [20]:
data.head(3)

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.7,33.5,8.5
1,2,닭갈비,NaN,NaN,NaN,400.0,595.61,44.9,45.9,25.8
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.1,45.5,31.6


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90608 entries, 0 to 90607
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FOOD_SEQ  90608 non-null  int64  
 1   음식명       90608 non-null  object 
 2   레시피       828 non-null    object 
 3   한중일양      828 non-null    object 
 4   밥국반메면     754 non-null    object 
 5   중량        90608 non-null  float64
 6   열량(kcal)  90608 non-null  float64
 7   탄수화물(g)   90608 non-null  float64
 8   단백질(g)    90608 non-null  float64
 9   지방(g)     90608 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 6.9+ MB


In [22]:
# Food_name 전처리하기
data['음식명'] = data['음식명'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))
data.head()

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,2,닭갈비,NaN,NaN,NaN,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90608 entries, 0 to 90607
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FOOD_SEQ  90608 non-null  int64  
 1   음식명       90608 non-null  object 
 2   레시피       828 non-null    object 
 3   한중일양      828 non-null    object 
 4   밥국반메면     754 non-null    object 
 5   중량        90608 non-null  float64
 6   열량(kcal)  90608 non-null  float64
 7   탄수화물(g)   90608 non-null  float64
 8   단백질(g)    90608 non-null  float64
 9   지방(g)     90608 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 6.9+ MB


In [24]:
# 혹시 모르니까 수틀리면 지우기
# cols = ['food_name']
# drop_index = food_p_p.loc[:,cols][food_p_p.loc[:,cols].duplicated()].index
# food_p_p.drop(drop_index, axis=0, inplace=True)

In [25]:
# 중복 확인
data.iloc[:,:7][data.duplicated(subset=['FOOD_SEQ','음식명'])]
# 이러면 중복이 없다는 소리네요~

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal)


In [26]:
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,2,닭갈비,NaN,NaN,NaN,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
...,...,...,...,...,...,...,...,...,...,...
90603,90604,"코코아,가루",NaN,NaN,NaN,100.0,268.00,43.63,22.85,20.06
90604,90605,"홍삼차,액상",NaN,NaN,NaN,100.0,40.00,9.27,0.28,0.24
90605,90606,"홍차,복숭아,가루",NaN,NaN,NaN,100.0,397.00,98.12,0.32,0.38
90606,90607,"마요네즈,저칼로리",NaN,NaN,NaN,100.0,341.00,12.35,0.89,31.97


In [27]:
# 안쓰기로 햇으요.
# # 표준치를 환산할 필요가 있을듯 합니다
# def normalized_food(df):
#   if df['Food_weigth'] >= 500:
#     df['Food_weigth'] = round(df['Food_weigth'] / df['Food_weigth'] * 250 , 2)
#     df['Food_calorie'] = round(df['Food_calorie'] / df['Food_weigth'] * 250 , 2)
#     df['Food_crb'] = round(df['Food_crb'] / df['Food_weigth'] * 250 , 2)
#     df['Food_protein'] = round(df['Food_protein'] / df['Food_weigth'] * 250 , 2)
#     df['Food_fat'] = round(df['Food_fat'] / df['Food_weigth'] * 250 , 2)
#   return df

In [28]:
data['레시피'].isna()

0        False
1         True
2        False
3        False
4        False
         ...  
90603     True
90604     True
90605     True
90606     True
90607     True
Name: 레시피, Length: 90608, dtype: bool

In [58]:
# DF내 모든 값을 비교해 비어있으면 삭제
# data.dropna(axis=0, how='any',inplace=True)

# 레시피가 nan값이 아닌 애들로만 구성.
data = data[data['레시피'].notna()]
# 인덱스 값 새롭게 지정하기
data = data.reset_index(drop=True)
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
2,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
3,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
4,6,도미구이,https://www.youtube.com/embed/Q0W5ag8ZbzU,한,메,250.0,397.00,2.10,61.10,16.50
...,...,...,...,...,...,...,...,...,...,...
823,7324,오징어튀김,https://www.youtube.com/embed/K7KCOvEKPbg,한,메,150.0,505.71,33.65,18.56,32.99
824,7325,채소튀김,https://www.youtube.com/embed/1uvcqAUjH8A,한,메,100.0,404.63,36.83,3.25,27.14
825,7326,치킨너겟,https://www.youtube.com/embed/aw1YhCxSF7c,한,메,150.0,468.40,14.93,22.47,35.42
826,7327,탕수육,https://www.youtube.com/embed/mzZNNMM2WuQ,중,NaN,200.0,358.97,24.84,27.21,16.75


미플린-지어(Mifflin-St.Jeor) 공식:

기본식: (10 × 몸무게) + (6.25 × 키) – (5 × 나이)

남성: 기본식 ＋ 5
여성: 기본식 － 161

기초대사량 ＋ 활동대사량 계산 식:

거의 운동하지 않음: 기초대사량 × 1.2
가벼운 운동(주 1~3일): 기초대사량 × 1.375
보통 수준: 기초대사량 × 1.55
적극적으로 운동(주 6~7일): 기초대사량 × 1.725
매우 적극적으로 운동(주 6~7일): 기초대사량 × 1.9

탄수화물: 1g당 4kcal
단백질: 1g당 4kcal
지방: 1g당 9kcal

SQL 구문을 만들어보아요
insert into member_info values (
  idman,
  12345,
  bususer,
  28,
  man,
  174.6,
  69.4,
  유지,
  1~3,


)

In [42]:
# 174 69 남성 유지 1.375 28

In [43]:
cal = ((10*69)+(6.25*174)-(5*28) +5)*1.375
cal

2258.4375

In [44]:
# cal을 55:25:20 으로 나눠야합니다.
crb = cal*11/20
protein = cal/5
fat =  cal/4
print(crb, protein, fat)

1242.140625 451.6875 564.609375


In [45]:
my_crb = crb/4
my_protein = protein/4
my_fat = fat/9
print(my_crb,my_protein,my_fat)

310.53515625 112.921875 62.734375


In [46]:
# 남성의 경우: BMR = 88.362 + (13.397 x 체중(kg)) + (4.799 x 키(cm)) - (5.677 x 나이(세))
new_cal = (88.362 + (13.397*69)+(4.799*174) - (5.677*28))
new_cal

1688.825

In [47]:
# 탄수화물: 1800kcal의 50-60%인 약 225-270g
# 단백질: 1800kcal의 15-20%인 약 68-90g
# 지방: 1800kcal의 20-30%인 약 40-60g

In [48]:
new_cal=new_cal*1.375
new_cal

2322.134375

In [49]:
new_crb = new_cal*0.5
new_protein = new_cal*0.15
new_fat = new_cal*0.35

In [50]:
new_crb = new_crb/4
new_protein = new_protein/4
new_fat = new_fat/9

In [51]:
print(new_crb,new_protein,new_fat)

290.266796875 87.0800390625 90.30522569444445


In [66]:
# 개인 정보에 기반한 활동 대사량, 기초대사량 구하기
# Physical Activity Level (PAL)
def user_PAL(weight, height, age, gender, activity, purpose):
  # 기초대사량 -> 키, 몸무게, 나이로 구성된 기본식
  base = (10*weight)+(6.25*height)-(5*age)
  # 남성일 경우
  if gender == "man":
    man_base=(base+5)
    # Eat-Fit을 쓰는 목적 -> 증량, 유지, 다이어트
    if purpose == "plus":
      man_base = man_base*1.2
      # 활동량 -> 1 : 아예 운동 X / 2 : 주 1~3 가벼운 운동 / 3 : 보통 수준 운동 / 4 : 적극적 운동 / 5: 매우 적극적 운동
      if activity == 1:
        man_base = man_base*1.2
      elif activity == 2:
        man_base = man_base*1.375
      elif activity == 3:
        man_base = man_base*1.55
      elif activity == 4:
        man_base = man_base*1.725
      elif activity == 5:
        man_base = man_base*1.9
    elif purpose == "maintain":
      man_base = man_base
      if activity == 1:
        man_base = man_base*1.2
      elif activity == 2:
        man_base = man_base*1.375
      elif activity == 3:
        man_base = man_base*1.55
      elif activity == 4:
        man_base = man_base*1.725
      elif activity == 5:
        man_base = man_base*1.9
    else :
      man_base = man_base*0.8
      if activity == 1:
        man_base = man_base*1.2
      elif activity == 2:
        man_base = man_base*1.375
      elif activity == 3:
        man_base = man_base*1.55
      elif activity == 4:
        man_base = man_base*1.725
      elif activity == 5:
        man_base = man_base*1.9

    crb=man_base*0.5
    protein = man_base*0.15
    fat = man_base*0.35

    man_crb = crb/4
    man_protein = protein/4
    man_fat = fat/9

    user = {}
    user = {'cal':man_base, 'crb':man_crb, 'protein':man_protein, 'fat':man_fat}
    return user

  else :
    woman_base = (base-161)
    if purpose == "plus":
      woman_base = woman_base*1.2
      if activity == 1:
        woman_base = woman_base*1.2
      elif activity == 2:
        woman_base = woman_base*1.375
      elif activity == 3:
        woman_base = woman_base*1.55
      elif activity == 4:
        woman_base = woman_base*1.725
      elif activity == 5:
        woman_base = woman_base*1.9
    elif purpose == "maintain":
      woman_base = woman_base
      if activity == 1:
        woman_base = woman_base*1.2
      elif activity == 2:
        woman_base = woman_base*1.375
      elif activity == 3:
        woman_base = woman_base*1.55
      elif activity == 4:
        woman_base = woman_base*1.725
      elif activity == 5:
        woman_base = woman_base*1.9
    else :
      woman_base = woman_base*0.8
      if activity == 1:
        woman_base = woman_base*1.2
      elif activity == 2:
        woman_base = woman_base*1.375
      elif activity == 3:
        woman_base = woman_base*1.55
      elif activity == 4:
        woman_base = woman_base*1.725
      elif activity == 5:
        woman_base = woman_base*1.9

      crb2=woman_base*0.5
      protein2 = woman_base*0.15
      fat2 = woman_base*0.35

      woman_crb = crb2/4
      woman_protein = protein2/4
      woman_fat = fat2/9

      user = {}
      user = {'cal':woman_base, 'crb':woman_crb, 'protein':woman_protein, 'fat':woman_fat}

      return user
  

In [67]:
#def user_PAL(weight, height, age, gender, activity, purpose):
user=user_PAL(69,174,28,'man',3,'maintain')
user

{'cal': 2545.875,
 'crb': 318.234375,
 'protein': 95.47031249999999,
 'fat': 99.00625}

In [68]:
user_cal=user['cal']
user_cal

2545.875

In [69]:
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
2,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
3,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
4,6,도미구이,https://www.youtube.com/embed/Q0W5ag8ZbzU,한,메,250.0,397.00,2.10,61.10,16.50
...,...,...,...,...,...,...,...,...,...,...
823,7324,오징어튀김,https://www.youtube.com/embed/K7KCOvEKPbg,한,메,150.0,505.71,33.65,18.56,32.99
824,7325,채소튀김,https://www.youtube.com/embed/1uvcqAUjH8A,한,메,100.0,404.63,36.83,3.25,27.14
825,7326,치킨너겟,https://www.youtube.com/embed/aw1YhCxSF7c,한,메,150.0,468.40,14.93,22.47,35.42
826,7327,탕수육,https://www.youtube.com/embed/mzZNNMM2WuQ,중,NaN,200.0,358.97,24.84,27.21,16.75


In [60]:
food_cal=data.loc[1, '열량(kcal)']
food_cal

558.47

In [70]:
user_cal = round(float(user_cal - food_cal),2)
user_cal

1987.4

In [71]:
# 내가 가진 영양정보에서 오늘 먹은 식단 제외하고 남은 영양 성분 구하는 공식
# 여기서 x는 먹은 음식의 food_seq입니다.
def remaining_nut(dic, df, x):
  # 여기서 dic은 상단의 user_PAL을 통해 만들어진 딕셔너리를 의미합니다
  # df 는 상단에서 변형한 data를 의미합니다!
  # df와 x를 넣는다면 후에 아래의 식처럼 보여줄 수 있습니다.

  user_cal=dic['cal']
  user_crb = dic['crb']
  user_protein = dic['protein']
  user_fat = dic['fat']

  food_cal = df.loc[x,'열량(kcal)']
  food_crb = df.loc[x,'탄수화물(g)']
  food_protein = df.loc[x,'단백질(g)']
  food_fat = df.loc[x,'지방(g)']

  remain_cal = round(float(user_cal-food_cal),3)
  remain_crb = round(float(user_crb-food_crb),3)
  remain_protein = round(float(user_protein-food_protein),3)
  remain_fat = round(float(user_fat-food_fat),3)

  user_dic = {}
  user_dic = {'cal':remain_cal, 'crb':remain_crb, 'protein':remain_protein, 'fat':remain_fat}

  return user_dic

In [72]:
# 식단을 먹고 남은 것을 기반으로 새로운 친구를 추출할 수 있지 않는가...
remain_user_data = remaining_nut(user, data, 1)
remain_user_data

{'cal': 1987.405, 'crb': 295.134, 'protein': 49.97, 'fat': 67.406}

In [73]:
remain_user_data['cal']

1987.405

In [74]:
# 현재 먹은 코드 => B,L,D
code = 'B'
# 이제 먹어야 할 코드 => B, L, D
now_code = 'L'

In [ ]:
# 나의 생각인데 식단을 추천할 것이라면??
# 함수를 만들지만 아침 점심 저녁을 확인해야 함.
# 아 점 저 로 확인한다면? (얘가 코드임)
# code = > 리스트나 딕셔너리 형태여야 합니다. why? 아,점을 먹엇다면? 이런 경우가 잇다
def recommend_dish(dic, df, code, now_code):
  # 여기서 dic은 사용자의 정보
  cal = dic['cal']
  crb = dic['crb']
  protein = dic['protein']
  fat = dic['fat']
  # df는 데이터베이스에 있는 친구 (영양정보데이터)
  food_cal = df.loc[x,'열량(kcal)']
  food_crb = df.loc[x,'탄수화물(g)']
  food_protein = df.loc[x,'단백질(g)']
  food_fat = df.loc[x,'지방(g)']

  # 아점저 코드를 받는다면? 
  # 아 : B  점 : L  저 : D
  # flask로 받는데 이름을 정해야 함. ==> 여기선 code로 갑시다.
  if 'B' in code and 'L' not in code and 'D' not in code:
    # 아침을 먹은 상황
    cal = cal/3
    crb = crb/3
    protein = protein/3
    fat = fat/3
    if now_code == 'L':
      # 점심에 추천할 음식 골라봅시다.
      
    return
  elif 'B' in code and 'L' in code and 'D' not in code:
    # 아침과 점심을 먹은 상황

    return
  elif 'B' not in code and 'L' not in code and 'D' not in code:
    # 아무것도 안먹었을 경우

    return
  elif 'B' not in code and 'L' in code and 'D' not in code:
    # 아침X 점심만 먹은 경우

    return
  else : 
    # 전혀 다른 경우

    return
    